In [1]:
import pandas as pd
import math
from geopy.distance import geodesic
from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
cur = conn.cursor()

In [3]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'sept'
año = 2023
print(f"{mes}{año}")

sept2023


In [4]:
# Obtener los ids de todos los viajes
cur.execute(f"SELECT DISTINCT tripid, uid FROM datos_uda_{mes}{año}")
# Obtener los resultados de la consulta
rows = cur.fetchall()
id_viajes = rows.copy()

In [5]:
len(id_viajes)

13301

In [6]:
def calc_bearing(lat1, lon1, lat2, lon2):
    # Convertir latitudes y longitudes a radianes
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calcular la diferencia de longitudes
    delta_lon = lon2_rad - lon1_rad
    
    # Calcular bearing usando la fórmula de la arcotangente
    y = math.sin(delta_lon) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon)
    bearing_rad = math.atan2(y, x)
    
    # Convertir bearing a grados y ajustar el rango a [0, 360)
    bearing_deg = math.degrees(bearing_rad)
    bearing_deg = (bearing_deg + 360) % 360
    
    return bearing_deg

In [7]:
# -----Agregar columnas para caracteristicas nuevas
consulta_sql = f'''DROP TABLE IF EXISTS caracteristicas_viajes_{mes}{año};
                  CREATE TABLE IF NOT EXISTS caracteristicas_viajes_{mes}{año}
                    (
                        tripid integer,
                        uid character varying(50),
                        latitude double precision,
                        longitude double precision,
                        distance double precision,
                        speed double precision,
                        aceleration double precision,
                        bearing double precision,
                        recorded_at timestamp without time zone
                    );
                '''
cur.execute(consulta_sql)
cur.execute(f"commit")

In [8]:
# --------------------Obtener caracteristicas de viajes
print(f"n_viajes: {len(id_viajes)}")
prog_anterior = 1 # En uno aun no procesa nada
id_viajes = id_viajes[prog_anterior - 1:]
prog = prog_anterior - 1
for id_viaje in id_viajes:
    prog += 1
    print(f"Progreso: {prog}", end="\r")
    
    # Obtener los ids de todos los viajes
    cur.execute(f"SELECT latitude, longitude, recorded_at FROM datos_uda_{mes}{año} where tripid='{id_viaje[0]}' and uid='{id_viaje[1]}' order by recorded_at")
    # Obtener los resultados de la consulta
    rows = cur.fetchall()
    puntos_viaje = rows.copy()

    # Crear un DataFrame a partir de la lista de tuplas
    df = pd.DataFrame(puntos_viaje, columns=['latitude', 'longitude', 'recorded_at'])

    # Calcular la distancia y el tiempo transcurrido entre puntos
    for i in range(0, len(df)):
        df.at[i, 'tripid'] = str(id_viaje[0])
        df.at[i, 'uid'] = id_viaje[1]      
        
        if i > 0:
            # Obtener las coordenadas de los puntos actual y anterior
            lat1, lon1, rec1 = df.at[i-1, 'latitude'], df.at[i-1, 'longitude'], df.at[i-1, 'recorded_at']
            lat2, lon2, rec2 = df.at[i, 'latitude'], df.at[i, 'longitude'], df.at[i, 'recorded_at']
            
            # Calcular la distancia entre los puntos actual y anterior
            distance = geodesic((lat1, lon1), (lat2, lon2)).meters
            df.at[i, 'distance (m)'] = distance    

            # Calcular el tiempo entre los puntos actual y anterior
            time_diff = (rec2 - rec1).total_seconds()
            df.at[i, 'time_diff (s)'] = time_diff

            # Calcular la velocidad (metros por segundo)
            speed = 0
            if df.at[i, 'time_diff (s)'] != 0:
                speed = df.at[i, 'distance (m)'] / df.at[i, 'time_diff (s)']
            df.at[i, 'speed (m/s)'] = abs(speed)

            # Calcular el bearing usando la fórmula del rumbo, pertenece al punto de partida
            df.at[i-1, 'bearing'] = calc_bearing(lat1, lon1, lat2, lon2)
            
            # Calcular la aceleración (metros por segundo cuadrado)
            if i > 1:
                # Calcular la diferencia de velocidad entre los puntos actual y anterior
                speed_diff = df.at[i, 'speed (m/s)'] - df.at[i-1, 'speed (m/s)']
                
                # Calcular la aceleración
                aceleration = speed_diff / time_diff if time_diff != 0 else 0
                
                # Asignar la aceleración al DataFrame
                df.at[i, 'aceleration (m/s^2)'] = aceleration
    
    # Definir el nuevo orden de las columnas
    new_columns_order = ['tripid', 'uid','latitude', 'longitude', 'distance (m)', 'speed (m/s)', 'aceleration (m/s^2)', 
                            'bearing', 'recorded_at']

    # Reorganizar las columnas del DataFrame
    df = df.reindex(columns=new_columns_order)
    for index, row in df.iterrows():
        consulta_sql = """
        INSERT INTO {} VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """.format(f'caracteristicas_viajes_{mes}{año}')

        # Ejecuta la consulta SQL con los valores de la fila actual
        cur.execute(consulta_sql, tuple(row))
    conn.commit()

n_viajes: 33044
